<a href="https://www.kaggle.com/code/kwodus/moj-nucla?scriptVersionId=282397314" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Averaging

### Averaging on one image

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt


def arccos_safe(x):
    return np.arccos(np.clip(x, -1.0, 1.0))

def normalize_to_rgb(coords):
    min_val = np.min(coords)
    max_val = np.max(coords)
    norm_coords = 255 * (coords - min_val) / (max_val - min_val + 1e-8)
    return norm_coords.astype(np.uint8)

def compute_motion_of_joints(skeleton_sequence):
    Sx = normalize_to_rgb(skeleton_sequence[:, :, 0])
    Sy = normalize_to_rgb(skeleton_sequence[:, :, 1])
    Sz = normalize_to_rgb(skeleton_sequence[:, :, 2])
    motion = np.stack([Sx, Sy, Sz], axis=2)   # (N,J,3)
    return motion

def compute_pose_orientation(skeleton_sequence):
    N, J, _ = skeleton_sequence.shape
    pose_orientation = np.zeros((N, J, 3))
    for t in range(N):
        for i in range(J):
            for j in range(J):
                if i == j:
                    continue
                diff = skeleton_sequence[t, i] - skeleton_sequence[t, j]
                norm = np.linalg.norm(diff) + 1e-8
                normalized = diff / norm
                pose_orientation[t, i] += arccos_safe(normalized)
            pose_orientation[t, i] /= (J - 1)
    return normalize_to_rgb(pose_orientation)

def compute_transition_orientation(skeleton_sequence):
    N, J, _ = skeleton_sequence.shape
    transition_orientation = np.zeros((N, J, 3))
    for t in range(1, N):
        for i in range(J):
            for j in range(J):
                if i == j:
                    continue
                diff = skeleton_sequence[t, i] - skeleton_sequence[t-1, j]
                norm = np.linalg.norm(diff) + 1e-8
                normalized = diff / norm
                transition_orientation[t, i] += arccos_safe(normalized)
            transition_orientation[t, i] /= (J - 1)
    return normalize_to_rgb(transition_orientation)

def build_moj_image_side_by_side(motion, pose, transition):
    N, J, _ = motion.shape

    pose_resized = cv2.resize(pose, (J, N), interpolation=cv2.INTER_CUBIC)
    transition_resized = cv2.resize(transition, (J, N), interpolation=cv2.INTER_CUBIC)

    PT_side = np.hstack([pose_resized, transition_resized])

    combined = np.hstack([motion, PT_side]).astype(np.float32)

    mn, mx = combined.min(), combined.max()
    if mx - mn < 1e-8:
        norm_img = np.zeros_like(combined, dtype=np.uint8)
    else:
        norm_img = 255.0 * (combined - mn) / (mx - mn)
        norm_img = np.clip(norm_img, 0, 255).astype(np.uint8)

    final = cv2.resize(norm_img, (224, 224), interpolation=cv2.INTER_CUBIC)
    return final



def test_one_sequence(seq_path, expected_joints=20):
    
    def load_skeleton_file(filepath):
        joints = []
        with open(filepath, 'r') as f:
            lines = f.read().strip().split('\n')
            for line in lines[1:]:
                if not line.strip(): continue
                parts = line.strip().split(',')
                if len(parts) < 3: continue
                x, y, z = float(parts[0]), float(parts[1]), float(parts[2])
                joints.append([x, y, z])
        return np.array(joints, dtype=np.float32)

    def load_sequence_skeletons(sequence_path, expected_joints):
        file_list_path = os.path.join(sequence_path, 'fileList.txt')
        frames = []
        with open(file_list_path, 'r') as f:
            for line in f:
                frame_id = line.strip().split()[0]
                skeleton_file = None
                for file in os.listdir(sequence_path):
                    if file.startswith(f"frame_{frame_id}_") and file.endswith("_skeletons.txt"):
                        skeleton_file = file
                        break
                if skeleton_file is None: continue
                joints = load_skeleton_file(os.path.join(sequence_path, skeleton_file))
                if joints.shape[0] == expected_joints:
                    frames.append(joints)
        return np.stack(frames) if frames else None

    skeleton_sequence = load_sequence_skeletons(seq_path, expected_joints)
    if skeleton_sequence is None:
        print("No skeletons found.")
        return

    motion = compute_motion_of_joints(skeleton_sequence)
    pose = compute_pose_orientation(skeleton_sequence)
    transition = compute_transition_orientation(skeleton_sequence)
    moj_image = build_moj_image_side_by_side(motion, pose, transition)


    print("Motion shape:", motion.shape)
    print("Pose shape:", pose.shape)
    print("Transition shape:", transition.shape)
    print("Final MOJ image shape:", moj_image.shape)

    fig, axes = plt.subplots(1, 4, figsize=(20,5))
    axes[0].imshow(motion); axes[0].set_title("Motion (S)"); axes[0].axis('off')
    axes[1].imshow(pose); axes[1].set_title("Pose (P)"); axes[1].axis('off')
    axes[2].imshow(transition); axes[2].set_title("Transition (T)"); axes[2].axis('off')
    axes[3].imshow(moj_image); axes[3].set_title("Final MOJ Image I=[S,[P,T]]"); axes[3].axis('off')
    plt.show()

seq_path = "/kaggle/input/n-ucla/multiview_action/view_1/a01_s01_e01"
test_one_sequence(seq_path, expected_joints=20)


### Averaging on all images

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from tqdm import tqdm


def arccos_safe(x):
    return np.arccos(np.clip(x, -1.0, 1.0))

def normalize_to_rgb(coords):
    min_val = np.min(coords)
    max_val = np.max(coords)
    norm_coords = 255 * (coords - min_val) / (max_val - min_val + 1e-8)
    return norm_coords.astype(np.uint8)

def compute_motion_of_joints(skeleton_sequence):
    Sx = normalize_to_rgb(skeleton_sequence[:, :, 0])
    Sy = normalize_to_rgb(skeleton_sequence[:, :, 1])
    Sz = normalize_to_rgb(skeleton_sequence[:, :, 2])
    motion = np.stack([Sx, Sy, Sz], axis=2)   # (N,J,3)
    return motion

def compute_pose_orientation(skeleton_sequence):
    N, J, _ = skeleton_sequence.shape
    pose_orientation = np.zeros((N, J, 3))
    for t in range(N):
        for i in range(J):
            for j in range(J):
                if i == j:
                    continue
                diff = skeleton_sequence[t, i] - skeleton_sequence[t, j]
                norm = np.linalg.norm(diff) + 1e-8
                normalized = diff / norm
                pose_orientation[t, i] += arccos_safe(normalized)
            pose_orientation[t, i] /= (J - 1)
    return normalize_to_rgb(pose_orientation)


def compute_transition_orientation(skeleton_sequence):
    N, J, _ = skeleton_sequence.shape
    transition_orientation = np.zeros((N, J, 3))
    for t in range(1, N):
        for i in range(J):
            for j in range(J):
                if i == j:
                    continue
                diff = skeleton_sequence[t, i] - skeleton_sequence[t-1, j]
                norm = np.linalg.norm(diff) + 1e-8
                normalized = diff / norm
                transition_orientation[t, i] += arccos_safe(normalized)
            transition_orientation[t, i] /= (J - 1)
    return normalize_to_rgb(transition_orientation)

def build_moj_image_side_by_side(motion, pose, transition):
    N, J, _ = motion.shape

    pose_resized = cv2.resize(pose, (J, N), interpolation=cv2.INTER_CUBIC)
    transition_resized = cv2.resize(transition, (J, N), interpolation=cv2.INTER_CUBIC)

    PT_side = np.hstack([pose_resized, transition_resized])

    combined = np.hstack([motion, PT_side]).astype(np.float32)

    mn, mx = combined.min(), combined.max()
    if mx - mn < 1e-8:
        norm_img = np.zeros_like(combined, dtype=np.uint8)
    else:
        norm_img = 255.0 * (combined - mn) / (mx - mn)
        norm_img = np.clip(norm_img, 0, 255).astype(np.uint8)

    
    final = cv2.resize(norm_img, (224, 224), interpolation=cv2.INTER_CUBIC)
    return final

def load_skeleton_file(filepath):
    joints = []
    with open(filepath, 'r') as f:
        lines = f.read().strip().split('\n')
        for line in lines[1:]:
            if not line.strip(): continue
            parts = line.strip().split(',')
            if len(parts) < 3: continue
            x, y, z = float(parts[0]), float(parts[1]), float(parts[2])
            joints.append([x, y, z])
    return np.array(joints, dtype=np.float32)

def load_sequence_skeletons(sequence_path, expected_joints=20):
    file_list_path = os.path.join(sequence_path, 'fileList.txt')
    frames = []
    with open(file_list_path, 'r') as f:
        for line in f:
            frame_id = line.strip().split()[0]
            skeleton_file = None
            for file in os.listdir(sequence_path):
                if file.startswith(f"frame_{frame_id}_") and file.endswith("_skeletons.txt"):
                    skeleton_file = file
                    break
            if skeleton_file is None: continue
            joints = load_skeleton_file(os.path.join(sequence_path, skeleton_file))
            if joints.shape[0] == expected_joints:
                frames.append(joints)
    return np.stack(frames) if frames else None

def process_nucla_dataset(base_path, output_dir):
    views_dir = os.path.join(base_path, 'multiview_action')
    os.makedirs(output_dir, exist_ok=True)

    moj_image_paths = []
    view_folders = [v for v in sorted(os.listdir(views_dir)) if os.path.isdir(os.path.join(views_dir, v))]

    for view_folder in view_folders:
        view_path = os.path.join(views_dir, view_folder)
        seq_folders = [s for s in sorted(os.listdir(view_path)) if os.path.isdir(os.path.join(view_path, s))]
        
        print(f"Processing view: {view_folder} ({len(seq_folders)} sequences)")
        for seq_folder in tqdm(seq_folders, desc=f"{view_folder} sequences"):
            seq_path = os.path.join(view_path, seq_folder)
            skeleton_sequence = load_sequence_skeletons(seq_path)
            if skeleton_sequence is None:
                continue

            motion = compute_motion_of_joints(skeleton_sequence)
            pose = compute_pose_orientation(skeleton_sequence)
            transition = compute_transition_orientation(skeleton_sequence)
            moj_image = build_moj_image_side_by_side(motion, pose, transition)

            save_path = os.path.join(output_dir, f"{view_folder}_{seq_folder}.png")
            cv2.imwrite(save_path, cv2.cvtColor(moj_image, cv2.COLOR_RGB2BGR))
            moj_image_paths.append(save_path)
    return moj_image_paths

def visualize_moj_images(image_paths, num_images=5):
    selected_paths = random.sample(image_paths, min(len(image_paths), num_images))
    plt.figure(figsize=(15, 3 * num_images))
    for i, path in enumerate(selected_paths):
        img = cv2.imread(path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(num_images, 1, i + 1)
        plt.imshow(img_rgb)
        plt.title(os.path.basename(path))
        plt.axis('off')
    plt.show()

base_path = '/kaggle/input/n-ucla'
output_moj_dir = '/kaggle/working/moj_images'

moj_image_paths = process_nucla_dataset(base_path, output_moj_dir)
visualize_moj_images(moj_image_paths, num_images=5)


### Model

#### Small Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers

def build_moj_cnn(input_shape=(224, 224, 3), num_classes=10, lr=0.0005):

    model = models.Sequential()

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same",
                            input_shape=input_shape,
                            kernel_initializer="he_normal",
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())


    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Flatten())

    model.add(layers.Dense(256, kernel_initializer="he_normal"))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(128, kernel_initializer="he_normal"))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.5))


    model.add(layers.Dense(num_classes, activation="softmax"))

    optimizer = optimizers.Adam(learning_rate=lr)

    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model


### Train and test using normal train test split

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

def load_moj_dataset(base_dir, target_size=(224,224)):
    X, y = [], []
    for fname in tqdm(os.listdir(base_dir), desc="Loading MOJ images"):
        if not fname.endswith(".png"):
            continue
        
        parts = fname.split("_")
        if len(parts) < 2:
            continue
        action_label = parts[1]   # "a01", "a02", ...

        img_path = os.path.join(base_dir, fname)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, target_size)

        X.append(img)
        y.append(action_label)
    
    return np.array(X, dtype=np.float32), np.array(y)


def train_on_nucla(moj_dir, num_classes=10, batch_size=64, epochs=30):
    
    X, y = load_moj_dataset(moj_dir, target_size=(224,224))
    X = X / 255.0  

    le = LabelEncoder()
    y_int = le.fit_transform(y)
    y_cat = to_categorical(y_int, num_classes=num_classes)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y_cat, test_size=0.2, random_state=42, stratify=y_cat
    )

    model= build_moj_cnn(input_shape=(224,224,3), num_classes=num_classes)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=10,
        batch_size=batch_size,
        verbose=1
    )

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f" Test Accuracy: {test_acc:.4f}")

    return model, history, le, x_test, y_test



moj_dir = "/kaggle/working/moj_images"  
model, history, label_encoder, x_test, y_test = train_on_nucla(moj_dir, num_classes=10, epochs=30)
model.summary()

from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)


cm = confusion_matrix(y_true, y_pred)


plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - MOJ CNN")
plt.show()


print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


#### Train and test using Leave one subject out (LOSO) training and testing

In [ ]:
import os
import re
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical


IMG_DIR = "/kaggle/working/moj_images"
IMG_SIZE = (224, 224)
NUM_CLASSES = 10   # number of action classes in dataset
EPOCHS = 10
BATCH_SIZE = 32


def parse_filename(filename):
    """
    Extract view, action, subject, execution from filename
    Example: view_1_a03_s10_e03.png
    """
    match = re.match(r"view_(\d+)_a(\d+)_s(\d+)_e(\d+)\.png", filename)
    if match:
        view, action, subject, execution = map(int, match.groups())
        return view, action, subject, execution
    return None

def build_action_map():
    """Build a mapping from actual action numbers to continuous 0..NUM_CLASSES-1"""
    action_ids = set()
    for fname in os.listdir(IMG_DIR):
        parsed = parse_filename(fname)
        if parsed:
            _, action, _, _ = parsed
            action_ids.add(action)
    action_ids = sorted(list(action_ids))
    action_map = {aid: idx for idx, aid in enumerate(action_ids)}
    return action_map

def load_dataset(img_dir=IMG_DIR, img_size=IMG_SIZE, action_map=None):
    X, y, subjects = [], [], []
    for fname in os.listdir(img_dir):
        if not fname.endswith(".png"):
            continue
        parsed = parse_filename(fname)
        if not parsed:
            continue
        view, action, subject, execution = parsed
        img_path = os.path.join(img_dir, fname)
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        img = img / 255.0
        X.append(img)
        y.append(action_map[action])  # map to 0..NUM_CLASSES-1
        subjects.append(subject)
    return np.array(X), np.array(y), np.array(subjects)


def run_single_subject_experiment(test_subject=1):
    # Build action mapping
    action_map = build_action_map()
    print("Action mapping:", action_map)

    X, y, subjects = load_dataset(action_map=action_map)

    
    train_idx = subjects != test_subject
    test_idx = subjects == test_subject

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    y_train = to_categorical(y_train, NUM_CLASSES)
    y_test = to_categorical(y_test, NUM_CLASSES)

    model = build_moj_cnn(input_shape=(224,224,3), num_classes=NUM_CLASSES)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=2
    )

    from sklearn.metrics import confusion_matrix, classification_report
    import seaborn as sns
    import matplotlib.pyplot as plt

    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"Confusion Matrix for subject {test_subject}")
    plt.show()

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))


    print(model.summary())
    
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nFinal Accuracy for subject {test_subject}: {acc:.4f}")
    return acc


acc = run_single_subject_experiment(test_subject=5)   


## With PoT2i

### Full dataset

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from tqdm import tqdm


def arccos_safe(x):
    return np.arccos(np.clip(x, -1.0, 1.0))

def normalize_to_rgb(coords):
    min_val = np.min(coords)
    max_val = np.max(coords)
    if max_val - min_val < 1e-8:
        return np.zeros_like(coords, dtype=np.uint8)
    norm_coords = 255.0 * (coords - min_val) / (max_val - min_val + 1e-8)
    return np.clip(norm_coords, 0, 255).astype(np.uint8)

def normalize_to_0_255(arr):
    a = arr.astype(np.float32)
    mn, mx = a.min(), a.max()
    if mx - mn < 1e-8:
        return np.zeros_like(a, dtype=np.uint8)
    scaled = 255.0 * (a - mn) / (mx - mn)
    return np.clip(scaled, 0, 255).astype(np.uint8)


def compute_motion_of_joints(skeleton_sequence):
    Sx = normalize_to_rgb(skeleton_sequence[:, :, 0])
    Sy = normalize_to_rgb(skeleton_sequence[:, :, 1])
    Sz = normalize_to_rgb(skeleton_sequence[:, :, 2])
    motion = np.stack([Sx, Sy, Sz], axis=2)  # (N,J,3)
    return motion


def get_pair_indices(J):
    return [(i, j) for i in range(J) for j in range(J) if i != j]

def compute_pairwise_pot2i_features(skeleton_sequence):
    """
    Returns:
      combined: (N, 4*M_pairs, 3) uint8
    """
    N, J, _ = skeleton_sequence.shape
    pairs = get_pair_indices(J)
    M = len(pairs)

    # Axes
    Ox = np.array([1.0, 0.0, 0.0], dtype=np.float32)
    Oy = np.array([0.0, 1.0, 0.0], dtype=np.float32)
    Oz = np.array([0.0, 0.0, 1.0], dtype=np.float32)

    # Containers
    pose_dists = np.zeros((N, M, 3), dtype=np.float32)
    pose_orients = np.zeros((N, M, 3), dtype=np.float32)
    trans_dists = np.zeros((N, M, 3), dtype=np.float32)
    trans_orients = np.zeros((N, M, 3), dtype=np.float32)

    for t in range(N):
        for idx, (i, j) in enumerate(pairs):
            # Pose vector
            u = skeleton_sequence[t, i] - skeleton_sequence[t, j]
            pose_dists[t, idx, :] = np.abs(u)  # projected distances δx,δy,δz
            norm_u = np.linalg.norm(u) + 1e-8
            u_unit = u / norm_u
            pose_orients[t, idx, 0] = arccos_safe(np.dot(u_unit, Ox))
            pose_orients[t, idx, 1] = arccos_safe(np.dot(u_unit, Oy))
            pose_orients[t, idx, 2] = arccos_safe(np.dot(u_unit, Oz))

            # Transition vector
            if t == 0:
                trans_dists[t, idx, :] = 0.0
                trans_orients[t, idx, :] = 0.0
            else:
                v = skeleton_sequence[t, i] - skeleton_sequence[t-1, j]
                trans_dists[t, idx, :] = np.abs(v)  # λx,λy,λz
                norm_v = np.linalg.norm(v) + 1e-8
                v_unit = v / norm_v
                trans_orients[t, idx, 0] = arccos_safe(np.dot(v_unit, Ox))
                trans_orients[t, idx, 1] = arccos_safe(np.dot(v_unit, Oy))
                trans_orients[t, idx, 2] = arccos_safe(np.dot(v_unit, Oz))

    # Normalize separately as per PoT2I
    pose_dists = normalize_to_0_255(pose_dists)
    trans_dists = normalize_to_0_255(trans_dists)
    pose_orients = normalize_to_0_255(pose_orients)
    trans_orients = normalize_to_0_255(trans_orients)

    # Arrange into pixels: [pose_dist, pose_orient, trans_dist, trans_orient] → 4*M width
    combined = np.concatenate([pose_dists, pose_orients, trans_dists, trans_orients], axis=0)  # (N, 4*M, 3)
    return combined

def make_PoT2I_image(skeleton_sequence, target_height, target_width):
    full = compute_pairwise_pot2i_features(skeleton_sequence)  # (4N, M, 3)
    H, W, _ = full.shape
    resized = cv2.resize(full, (target_width, target_height), interpolation=cv2.INTER_CUBIC)
    if resized.dtype != np.uint8:
        resized = np.clip(resized, 0, 255).astype(np.uint8)
    return resized  # (target_height, target_width, 3)


def concatenate_motion_and_pot(motion, pot_resized):
    N, J, _ = motion.shape
    # Ensure same height
    H = motion.shape[0]
    W = motion.shape[1]

    # Resize pot to match motion height
    pot_resized = cv2.resize(pot_resized, (W, H), interpolation=cv2.INTER_CUBIC)

    # Concatenate horizontally (left-right)
    combined = np.concatenate([motion, pot_resized], axis=1)  # shape (H, 2*W, 3)

    # Normalize to [0,255]
    mn, mx = combined.min(), combined.max()
    if mx - mn < 1e-8:
        norm_img = np.zeros_like(combined, dtype=np.uint8)
    else:
        norm_img = 255.0 * (combined - mn) / (mx - mn)
        norm_img = np.clip(norm_img, 0, 255).astype(np.uint8)

    # Resize final to 224x224
    final = cv2.resize(norm_img, (224, 224), interpolation=cv2.INTER_CUBIC)
    return final


def load_skeleton_file(filepath):
    joints = []
    with open(filepath, 'r') as f:
        lines = f.read().strip().split('\n')
        for line in lines[1:]:
            if not line.strip():
                continue
            parts = line.strip().split(',')
            if len(parts) < 3:
                continue
            x, y, z = float(parts[0]), float(parts[1]), float(parts[2])
            joints.append([x, y, z])
    return np.array(joints, dtype=np.float32)

def compute_pot2i_blocks(skeleton_sequence):
    N, J, _ = skeleton_sequence.shape
    pairs = get_pair_indices(J)
    M = len(pairs)

    Ox, Oy, Oz = np.eye(3, dtype=np.float32)

    pose_dists = np.zeros((N, M, 3), dtype=np.float32)
    pose_orients = np.zeros((N, M, 3), dtype=np.float32)
    trans_dists = np.zeros((N, M, 3), dtype=np.float32)
    trans_orients = np.zeros((N, M, 3), dtype=np.float32)

    for t in range(N):
        for idx, (i, j) in enumerate(pairs):
            u = skeleton_sequence[t, i] - skeleton_sequence[t, j]
            pose_dists[t, idx, :] = np.abs(u)
            norm_u = np.linalg.norm(u) + 1e-8
            u_unit = u / norm_u
            pose_orients[t, idx, :] = [arccos_safe(np.dot(u_unit, Ox)),
                                       arccos_safe(np.dot(u_unit, Oy)),
                                       arccos_safe(np.dot(u_unit, Oz))]
            if t > 0:
                v = skeleton_sequence[t, i] - skeleton_sequence[t-1, j]
                trans_dists[t, idx, :] = np.abs(v)
                norm_v = np.linalg.norm(v) + 1e-8
                v_unit = v / norm_v
                trans_orients[t, idx, :] = [arccos_safe(np.dot(v_unit, Ox)),
                                            arccos_safe(np.dot(v_unit, Oy)),
                                            arccos_safe(np.dot(v_unit, Oz))]

    # Normalize
    pose_dists = normalize_to_0_255(pose_dists)
    pose_orients = normalize_to_0_255(pose_orients)
    trans_dists = normalize_to_0_255(trans_dists)
    trans_orients = normalize_to_0_255(trans_orients)

    return pose_dists, pose_orients, trans_dists, trans_orients


def load_sequence_skeletons(sequence_path, expected_joints=20):
    file_list_path = os.path.join(sequence_path, 'fileList.txt')
    frames = []
    if not os.path.exists(file_list_path):
        return None
    with open(file_list_path, 'r') as f:
        for line in f:
            frame_id = line.strip().split()[0]
            skeleton_file = None
            for file in os.listdir(sequence_path):
                if file.startswith(f"frame_{frame_id}_") and file.endswith("_skeletons.txt"):
                    skeleton_file = file
                    break
            if skeleton_file is None:
                continue
            joints = load_skeleton_file(os.path.join(sequence_path, skeleton_file))
            if joints.shape[0] == expected_joints:
                frames.append(joints)
    if not frames:
        return None
    return np.stack(frames)


def process_nucla_dataset(base_path, output_dir, expected_joints=20):
    views_dir = os.path.join(base_path, 'multiview_action')
    os.makedirs(output_dir, exist_ok=True)
    moj_image_paths = []

    view_folders = [v for v in sorted(os.listdir(views_dir)) if os.path.isdir(os.path.join(views_dir, v))]
    for view_folder in view_folders:
        view_path = os.path.join(views_dir, view_folder)
        seq_folders = [s for s in sorted(os.listdir(view_path)) if os.path.isdir(os.path.join(view_path, s))]

        print(f"Processing view: {view_folder} ({len(seq_folders)} sequences)")
        for seq_folder in tqdm(seq_folders, desc=f"{view_folder} sequences"):
            seq_path = os.path.join(view_path, seq_folder)
            skeleton_sequence = load_sequence_skeletons(seq_path, expected_joints=expected_joints)
            if skeleton_sequence is None:
                continue

            motion = compute_motion_of_joints(skeleton_sequence)
            pot_resized = make_PoT2I_image(
                skeleton_sequence,
                target_height=motion.shape[0],  # match motion height (N)
                target_width=motion.shape[1]    # match motion width (J)
            )
            moj_image = concatenate_motion_and_pot(motion, pot_resized)


            save_path = os.path.join(output_dir, f"{view_folder}_{seq_folder}.png")
            cv2.imwrite(save_path, cv2.cvtColor(moj_image, cv2.COLOR_RGB2BGR))
            moj_image_paths.append(save_path)

    return moj_image_paths


def visualize_moj_images(image_paths, num_images=5):
    selected_paths = random.sample(image_paths, min(len(image_paths), num_images))
    plt.figure(figsize=(15, 3 * num_images))
    for i, path in enumerate(selected_paths):
        img = cv2.imread(path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(num_images, 1, i + 1)
        plt.imshow(img_rgb)
        plt.title(os.path.basename(path))
        plt.axis('off')
    plt.show()


if __name__ == "__main__":
    base_path = '/kaggle/input/n-ucla'
    output_moj_dir = '/kaggle/working/moj_images'
    moj_image_paths = process_nucla_dataset(base_path, output_moj_dir, expected_joints=20)
    if moj_image_paths:
        visualize_moj_images(moj_image_paths, num_images=5)


### Single example

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt


def arccos_safe(x):
    return np.arccos(np.clip(x, -1.0, 1.0))

def normalize_to_rgb(coords):
    min_val = np.min(coords)
    max_val = np.max(coords)
    if max_val - min_val < 1e-8:
        return np.zeros_like(coords, dtype=np.uint8)
    norm_coords = 255.0 * (coords - min_val) / (max_val - min_val + 1e-8)
    return np.clip(norm_coords, 0, 255).astype(np.uint8)

def normalize_to_0_255(arr):
    a = arr.astype(np.float32)
    mn, mx = a.min(), a.max()
    if mx - mn < 1e-8:
        return np.zeros_like(a, dtype=np.uint8)
    scaled = 255.0 * (a - mn) / (mx - mn)
    return np.clip(scaled, 0, 255).astype(np.uint8)


def compute_motion_of_joints(skeleton_sequence):
    Sx = normalize_to_rgb(skeleton_sequence[:, :, 0])
    Sy = normalize_to_rgb(skeleton_sequence[:, :, 1])
    Sz = normalize_to_rgb(skeleton_sequence[:, :, 2])
    motion = np.stack([Sx, Sy, Sz], axis=2)  # (N,J,3)
    return motion


def get_pair_indices(J):
    return [(i, j) for i in range(J) for j in range(J) if i != j]

def compute_pairwise_pot2i_features(skeleton_sequence):
    N, J, _ = skeleton_sequence.shape
    pairs = get_pair_indices(J)
    M = len(pairs)

    Ox = np.array([1.0, 0.0, 0.0], dtype=np.float32)
    Oy = np.array([0.0, 1.0, 0.0], dtype=np.float32)
    Oz = np.array([0.0, 0.0, 1.0], dtype=np.float32)

    pose_dists = np.zeros((N, M, 3), dtype=np.float32)
    pose_orients = np.zeros((N, M, 3), dtype=np.float32)
    trans_dists = np.zeros((N, M, 3), dtype=np.float32)
    trans_orients = np.zeros((N, M, 3), dtype=np.float32)

    for t in range(N):
        for idx, (i, j) in enumerate(pairs):
            u = skeleton_sequence[t, i] - skeleton_sequence[t, j]
            pose_dists[t, idx, :] = np.abs(u)
            norm_u = np.linalg.norm(u) + 1e-8
            u_unit = u / norm_u
            pose_orients[t, idx, :] = [
                arccos_safe(np.dot(u_unit, Ox)),
                arccos_safe(np.dot(u_unit, Oy)),
                arccos_safe(np.dot(u_unit, Oz))
            ]
            if t > 0:
                v = skeleton_sequence[t, i] - skeleton_sequence[t-1, j]
                trans_dists[t, idx, :] = np.abs(v)
                norm_v = np.linalg.norm(v) + 1e-8
                v_unit = v / norm_v
                trans_orients[t, idx, :] = [
                    arccos_safe(np.dot(v_unit, Ox)),
                    arccos_safe(np.dot(v_unit, Oy)),
                    arccos_safe(np.dot(v_unit, Oz))
                ]

    pose_dists = normalize_to_0_255(pose_dists)
    trans_dists = normalize_to_0_255(trans_dists)
    pose_orients = normalize_to_0_255(pose_orients)
    trans_orients = normalize_to_0_255(trans_orients)

    combined = np.concatenate([pose_dists, pose_orients, trans_dists, trans_orients], axis=0)
    return combined

def make_PoT2I_image(skeleton_sequence, target_height, target_width):
    full = compute_pairwise_pot2i_features(skeleton_sequence)
    resized = cv2.resize(full, (target_width, target_height), interpolation=cv2.INTER_CUBIC)
    return np.clip(resized, 0, 255).astype(np.uint8)


def concatenate_motion_and_pot(motion, pot_resized):
    H, W, _ = motion.shape
    pot_resized = cv2.resize(pot_resized, (W, H), interpolation=cv2.INTER_CUBIC)
    combined = np.concatenate([motion, pot_resized], axis=1)  # left-right

    mn, mx = combined.min(), combined.max()
    if mx - mn < 1e-8:
        norm_img = np.zeros_like(combined, dtype=np.uint8)
    else:
        norm_img = 255.0 * (combined - mn) / (mx - mn)
        norm_img = np.clip(norm_img, 0, 255).astype(np.uint8)

    final = cv2.resize(norm_img, (224, 224), interpolation=cv2.INTER_CUBIC)
    return final


def load_skeleton_file(filepath):
    joints = []
    with open(filepath, 'r') as f:
        lines = f.read().strip().split('\n')
        for line in lines[1:]:
            if not line.strip():
                continue
            parts = line.strip().split(',')
            if len(parts) < 3:
                continue
            x, y, z = float(parts[0]), float(parts[1]), float(parts[2])
            joints.append([x, y, z])
    return np.array(joints, dtype=np.float32)

def load_sequence_skeletons(sequence_path, expected_joints=20):
    file_list_path = os.path.join(sequence_path, 'fileList.txt')
    frames = []
    if not os.path.exists(file_list_path):
        return None
    with open(file_list_path, 'r') as f:
        for line in f:
            frame_id = line.strip().split()[0]
            skeleton_file = None
            for file in os.listdir(sequence_path):
                if file.startswith(f"frame_{frame_id}_") and file.endswith("_skeletons.txt"):
                    skeleton_file = file
                    break
            if skeleton_file is None:
                continue
            joints = load_skeleton_file(os.path.join(sequence_path, skeleton_file))
            if joints.shape[0] == expected_joints:
                frames.append(joints)
    if not frames:
        return None
    return np.stack(frames)


if __name__ == "__main__":
    base_path = '/kaggle/input/n-ucla'
    output_moj_dir = '/kaggle/working/moj_images'
    os.makedirs(output_moj_dir, exist_ok=True)

    # Pick one view & one sequence
    view_folder = sorted(os.listdir(os.path.join(base_path, "multiview_action")))[0]
    seq_folder = sorted(os.listdir(os.path.join(base_path, "multiview_action", view_folder)))[0]

    seq_path = os.path.join(base_path, "multiview_action", view_folder, seq_folder)
    skeleton_sequence = load_sequence_skeletons(seq_path, expected_joints=20)

    if skeleton_sequence is not None:
        motion = compute_motion_of_joints(skeleton_sequence)
        pot_resized = make_PoT2I_image(skeleton_sequence, target_height=motion.shape[0], target_width=motion.shape[1])
        moj_image = concatenate_motion_and_pot(motion, pot_resized)

        
        save_path = os.path.join(output_moj_dir, f"{view_folder}_{seq_folder}.png")
        cv2.imwrite(save_path, cv2.cvtColor(moj_image, cv2.COLOR_RGB2BGR))

        print("Saved test image:", save_path)

        plt.imshow(moj_image)
        plt.title("MOJ + PoT2I (Side-by-Side)")
        plt.axis("off")
        plt.show()
    else:
        print("No valid skeleton sequence found.")


### MOdel

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers

def build_moj_cnn(input_shape=(224, 224, 3), num_classes=10, lr=0.0005):
    """
    Build the proposed lightweight CNN for MOJ descriptors.
    
    Args:
        input_shape: tuple, input image shape (default 224x224x3)
        num_classes: int, number of action classes
        lr: float, learning rate (default 0.0005)

    Returns:
        model: compiled Keras model
    """

    model = models.Sequential()

   
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same",
                            input_shape=input_shape,
                            kernel_initializer="he_normal",
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    
    model.add(layers.Flatten())

   
    model.add(layers.Dense(256, kernel_initializer="he_normal"))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(128, kernel_initializer="he_normal"))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.5))

   
    model.add(layers.Dense(num_classes, activation="softmax"))

    optimizer = optimizers.Adam(learning_rate=lr)

    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model


### Normal Train test split

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm


def load_moj_dataset(base_dir, target_size=(224,224)):
    X, y = [], []
    for fname in tqdm(os.listdir(base_dir), desc="Loading MOJ images"):
        if not fname.endswith(".png"):
            continue
        
        # Example: view_1_a01_s01_e01.png → action = a01
        parts = fname.split("_")
        if len(parts) < 2:
            continue
        action_label = parts[1]   # "a01", "a02", ...

        img_path = os.path.join(base_dir, fname)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, target_size)

        X.append(img)
        y.append(action_label)
    
    return np.array(X, dtype=np.float32), np.array(y)

def train_on_nucla(moj_dir, num_classes=10, batch_size=64, epochs=30):
    # Load dataset
    X, y = load_moj_dataset(moj_dir, target_size=(224,224))
    X = X / 255.0  # normalize

    # Encode labels
    le = LabelEncoder()
    y_int = le.fit_transform(y)
    y_cat = to_categorical(y_int, num_classes=num_classes)

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_cat, test_size=0.2, random_state=42, stratify=y_cat
    )

    # Build CNN (your function from above)
    model= build_moj_cnn(input_shape=(224,224,3), num_classes=num_classes)

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    # Evaluate
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f" Test Accuracy: {test_acc:.4f}")

    return model, history, le, x_test, y_test


moj_dir = "/kaggle/working/moj_images"  
model, history, label_encoder, x_test, y_test = train_on_nucla(moj_dir, num_classes=10, epochs=10)
print(model.summary(
    \
))
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)


cm = confusion_matrix(y_true, y_pred)


plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - MOJ CNN")
plt.show()


print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))



## Flattening all of them


### entire dataset

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from tqdm import tqdm


def arccos_safe(x):
    return np.arccos(np.clip(x, -1.0, 1.0))

def normalize_to_rgb(coords):
    min_val = np.min(coords)
    max_val = np.max(coords)
    norm_coords = 255 * (coords - min_val) / (max_val - min_val + 1e-8)
    return norm_coords.astype(np.uint8)


def compute_motion_of_joints(skeleton_sequence):
    Sx = normalize_to_rgb(skeleton_sequence[:, :, 0])
    Sy = normalize_to_rgb(skeleton_sequence[:, :, 1])
    Sz = normalize_to_rgb(skeleton_sequence[:, :, 2])
    return np.stack([Sx, Sy, Sz], axis=2)   # (N,J,3)

def compute_pose_orientation(skeleton_sequence):
    N, J, _ = skeleton_sequence.shape
    pose_orientation = []
    for t in range(N):
        frame_orients = []
        for i in range(J):
            for j in range(J):
                if i == j: continue
                diff = skeleton_sequence[t, i] - skeleton_sequence[t, j]
                norm = np.linalg.norm(diff) + 1e-8
                normalized = diff / norm
                frame_orients.append(arccos_safe(normalized))
        pose_orientation.append(frame_orients)
    pose_orientation = np.array(pose_orientation)  # (N, J*(J-1), 3)
    return normalize_to_rgb(pose_orientation)

def compute_transition_orientation(skeleton_sequence):
    N, J, _ = skeleton_sequence.shape
    if N <= 1:
        # Not enough frames for transition
        return np.zeros((N, J*(J-1), 3), dtype=np.uint8)

    trans_orientation = []
    for t in range(1, N):
        frame_orients = []
        for i in range(J):
            for j in range(J):
                if i == j: continue
                diff = skeleton_sequence[t, i] - skeleton_sequence[t-1, j]
                norm = np.linalg.norm(diff) + 1e-8
                normalized = diff / norm
                frame_orients.append(arccos_safe(normalized))
        trans_orientation.append(frame_orients)
    trans_orientation = np.array(trans_orientation)  # (N-1, J*(J-1), 3)
    
    # Pad the first frame with zeros to match motion/pose frame count
    first_frame = np.zeros_like(trans_orientation[0:1])
    trans_orientation = np.vstack([first_frame, trans_orientation])
    return normalize_to_rgb(trans_orientation)



def build_action_image(motion, pose, transition, target_size=(224,224)):
    # Ensure same frame count
    N = min(motion.shape[0], pose.shape[0], transition.shape[0])
    motion, pose, transition = motion[:N], pose[:N], transition[:N]

    frames = []
    for t in range(N):
        moj_row = motion[t].flatten()
        pose_row = pose[t].flatten()
        trans_row = transition[t].flatten()
        frame_row = np.concatenate([moj_row, pose_row, trans_row], axis=-1)
        frames.append(frame_row)
    feature_matrix = np.stack(frames)  # (N, F)

    # Normalize
    feature_matrix = (feature_matrix - feature_matrix.min()) / (feature_matrix.max() - feature_matrix.min() + 1e-8)
    feature_matrix = (feature_matrix * 255).astype(np.uint8)

    # Reshape into RGB image
    W = feature_matrix.shape[1] // 3
    img = feature_matrix.reshape(N, W, 3)

    # Resize to CNN input size
    img_resized = cv2.resize(img, target_size)
    return img_resized


def load_skeleton_file(filepath):
    joints = []
    with open(filepath, 'r') as f:
        lines = f.read().strip().split('\n')
        for line in lines[1:]:
            if not line.strip(): continue
            parts = line.strip().split(',')
            if len(parts) < 3: continue
            x, y, z = float(parts[0]), float(parts[1]), float(parts[2])
            joints.append([x, y, z])
    return np.array(joints, dtype=np.float32)

def load_sequence_skeletons(sequence_path, expected_joints=20):
    file_list_path = os.path.join(sequence_path, 'fileList.txt')
    frames = []
    with open(file_list_path, 'r') as f:
        for line in f:
            frame_id = line.strip().split()[0]
            skeleton_file = None
            for file in os.listdir(sequence_path):
                if file.startswith(f"frame_{frame_id}_") and file.endswith("_skeletons.txt"):
                    skeleton_file = file
                    break
            if skeleton_file is None: continue
            joints = load_skeleton_file(os.path.join(sequence_path, skeleton_file))
            if joints.shape[0] == expected_joints:
                frames.append(joints)
    return np.stack(frames) if frames else None

def process_nucla_dataset(base_path, output_dir):
    views_dir = os.path.join(base_path, 'multiview_action')
    os.makedirs(output_dir, exist_ok=True)

    image_paths = []
    view_folders = [v for v in sorted(os.listdir(views_dir)) if os.path.isdir(os.path.join(views_dir, v))]

    for view_folder in view_folders:
        view_path = os.path.join(views_dir, view_folder)
        seq_folders = [s for s in sorted(os.listdir(view_path)) if os.path.isdir(os.path.join(view_path, s))]
        
        print(f"Processing view: {view_folder} ({len(seq_folders)} sequences)")
        for seq_folder in tqdm(seq_folders, desc=f"{view_folder} sequences"):
            seq_path = os.path.join(view_path, seq_folder)
            skeleton_sequence = load_sequence_skeletons(seq_path)
            if skeleton_sequence is None:
                continue

            motion = compute_motion_of_joints(skeleton_sequence)
            pose = compute_pose_orientation(skeleton_sequence)
            transition = compute_transition_orientation(skeleton_sequence)
            img = build_action_image(motion, pose, transition)

            save_path = os.path.join(output_dir, f"{view_folder}_{seq_folder}.png")
            cv2.imwrite(save_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
            image_paths.append(save_path)
    return image_paths

def visualize_images(image_paths, num_images=5):
    selected_paths = random.sample(image_paths, min(len(image_paths), num_images))
    plt.figure(figsize=(15, 3 * num_images))
    for i, path in enumerate(selected_paths):
        img = cv2.imread(path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(num_images, 1, i + 1)
        plt.imshow(img_rgb)
        plt.title(os.path.basename(path))
        plt.axis('off')
    plt.show()

base_path = '/kaggle/input/n-ucla'
output_dir = '/kaggle/working/action_images'

image_paths = process_nucla_dataset(base_path, output_dir)
visualize_images(image_paths, num_images=5)


### One image

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Pick any one image (first image for example)
img_path = image_paths[0]

# Load and convert to RGB
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Plot
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.title(img_path.split('/')[-1])
plt.axis('off')
plt.show()


### MOdel

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers

def build_moj_cnn(input_shape=(224, 224, 3), num_classes=10, lr=0.0005):
    """
    Build the proposed lightweight CNN for MOJ descriptors.

    Args:
        input_shape: tuple, input image shape (default 224x224x3)
        num_classes: int, number of action classes
        lr: float, learning rate (default 0.0005)

    Returns:
        model: compiled Keras model
    """

    model = models.Sequential()

   
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same",
                            input_shape=input_shape,
                            kernel_initializer="he_normal",
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())


    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same",
                            kernel_initializer="he_normal",
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Flatten())

    model.add(layers.Dense(256, kernel_initializer="he_normal"))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(128, kernel_initializer="he_normal"))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.5))


    model.add(layers.Dense(num_classes, activation="softmax"))


    optimizer = optimizers.Adam(learning_rate=lr)


    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model


### Train and test

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay   ### >>> ADDED LINE


def load_moj_dataset(base_dir, target_size=(224,224)):
    X, y = [], []
    for fname in tqdm(os.listdir(base_dir), desc="Loading MOJ images"):
        if not fname.endswith(".png"):
            continue
        
        parts = fname.split("_")
        if len(parts) < 2:
            continue
        action_label = parts[1]   # "a01"

        img_path = os.path.join(base_dir, fname)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, target_size)

        X.append(img)
        y.append(action_label)
    
    return np.array(X, dtype=np.float32), np.array(y)


def train_on_nucla(moj_dir, num_classes=10, batch_size=64, epochs=30):
    X, y = load_moj_dataset(moj_dir, target_size=(224,224))
    X = X / 255.0  

    le = LabelEncoder()
    y_int = le.fit_transform(y)
    y_cat = to_categorical(y_int, num_classes=num_classes)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y_cat, test_size=0.2, random_state=42, stratify=y_cat
    )

    model = build_moj_cnn(input_shape=(224,224,3), num_classes=num_classes)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f" Test Accuracy: {test_acc:.4f}")

    model.summary()   
    
    y_pred_probs = model.predict(X_test)                    
    y_pred = np.argmax(y_pred_probs, axis=1)                
    y_true = np.argmax(y_test, axis=1)                      
    class_names = le.classes_                               

    cm = confusion_matrix(y_true, y_pred)                   
    plt.figure(figsize=(10, 8))                             
    disp = ConfusionMatrixDisplay(cm, display_labels=class_names)  
    disp.plot(xticks_rotation=45, cmap="Blues")             
    plt.title("NUCLA MOJ Confusion Matrix")                 
    plt.show()                                             
    

    return model, history, le


moj_dir = "/kaggle/working/action_images"
model, history, label_encoder = train_on_nucla(moj_dir, num_classes=10, epochs=30)


### LOSO

In [ ]:
import os
import re
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

IMG_DIR = "/kaggle/working/action_images"
IMG_SIZE = (224, 224)
NUM_CLASSES = 10   # number of action classes in dataset
EPOCHS = 10
BATCH_SIZE = 32


def parse_filename(filename):
    """
    Extract view, action, subject, execution from filename
    Example: view_1_a03_s10_e03.png
    """
    match = re.match(r"view_(\d+)_a(\d+)_s(\d+)_e(\d+)\.png", filename)
    if match:
        view, action, subject, execution = map(int, match.groups())
        return view, action, subject, execution
    return None

def build_action_map():
    """Build a mapping from actual action numbers to continuous 0..NUM_CLASSES-1"""
    action_ids = set()
    for fname in os.listdir(IMG_DIR):
        parsed = parse_filename(fname)
        if parsed:
            _, action, _, _ = parsed
            action_ids.add(action)
    action_ids = sorted(list(action_ids))
    action_map = {aid: idx for idx, aid in enumerate(action_ids)}
    return action_map

def load_dataset(img_dir=IMG_DIR, img_size=IMG_SIZE, action_map=None):
    X, y, subjects = [], [], []
    for fname in os.listdir(img_dir):
        if not fname.endswith(".png"):
            continue
        parsed = parse_filename(fname)
        if not parsed:
            continue
        view, action, subject, execution = parsed
        img_path = os.path.join(img_dir, fname)
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        img = img / 255.0
        X.append(img)
        y.append(action_map[action])  # map to 0..NUM_CLASSES-1
        subjects.append(subject)
    return np.array(X), np.array(y), np.array(subjects)


def run_single_subject_experiment(test_subject=1):
    # Build action mapping
    action_map = build_action_map()
    print("Action mapping:", action_map)

    X, y, subjects = load_dataset(action_map=action_map)

    # Split by subject
    train_idx = subjects != test_subject
    test_idx = subjects == test_subject

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # One-hot encode labels
    y_train = to_categorical(y_train, NUM_CLASSES)
    y_test = to_categorical(y_test, NUM_CLASSES)

    # Build fresh model
    model = build_moj_cnn(input_shape=(224,224,3), num_classes=NUM_CLASSES)

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=2
    )

    # Evaluate
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nFinal Accuracy for subject {test_subject}: {acc:.4f}")
    return acc

acc = run_single_subject_experiment(test_subject=5)   
